In [ ]:
import sqlite3
import collections
import json


data = json.load(open("mana.json"))

In [ ]:
def generate_mana_id(month, day, title, verse):
    return str(month) + "-" + str(day) + "_" + str(title).strip().lower().replace(" ", "") + "_" + str(verse).strip().replace(" ", "").lower()

class Mana:
    def __init__(self, columns):
        for col in columns:
            self.__setattr__(col, "")

    def insert_into_db(self, database_path):
        not_other = ["id", "month", "vakiteny", "lohateny", "content", "f", "day"]
        con = sqlite3.connect(database_path)
        cur = con.cursor()
        cur.execute("""
            CREATE TABLE IF NOT EXISTS mana (
            id varchar(100) PRIMARY KEY,
            month int NOT NULL,
            day int NOT NULL,
            vakiteny varchar(50),
            lohateny varchar(50),
            content text,
            f varchar(10),
            other_as_json text);""")

        query = "insert into mana "
        map_ = {"columns":"(", "values":" values ("}

        other = {} ## other_as_json = json.dump(other)
        for (attr_name, value) in self.__dict__.items():
            formatted_value = value
            if attr_name.lower().strip() in not_other:
                if isinstance(value, list):
                    formatted_value = ""
                    for item in value:
                        formatted_value += str(item).replace("\"", "'") + " "
                    formatted_value = "\"" + formatted_value.strip() + "\""

                elif isinstance(value, str):
                    try:
                        formatted_value = int(value)
                    except:
                        formatted_value = "\"" + value.strip() + "\""
                map_["columns"] += attr_name + ", "
                map_["values"] += str(formatted_value) + ", "
            else:
                other.update({attr_name.lower().strip(): str(value)})

        other = json.dumps(other).replace("\"", "'")
        map_["columns"] += "other_as_json" + ")"
        map_["values"] += "\""+ str(other) + "\"" + ")"

        query += map_["columns"] + map_["values"]
                
        cur.execute(query)
        con.commit()
        con.close()
        

    def describe(self):
        for (attr_name, value) in self.__dict__.items():
            print(attr_name, "=>", value)

In [ ]:
# columns = []
# for (month, item) in data.items():
#     for (day_of_month, item) in item.items():
#         for (key, value) in item.items():
#             columns.append(key.lower().strip())

# columns = sorted(list(set(columns)))

for (month, item) in data.items():
    for (day_of_month, item) in item.items():
        mana = Mana([])
        mana.__setattr__("month", month)
        mana.__setattr__("day", day_of_month)
        for (key, value) in item.items():
            mana.__setattr__(key.lower().strip(), value)
        mana.__setattr__("id", generate_mana_id(month, day_of_month, item["lohateny"], item["vakiteny"]))
        mana.insert_into_db("baiboly.db")